In [4]:
%%bash

#Default projection (WA state plane north)
#http://spatialreference.org/ref/esri/nad-1983-stateplane-washington-south-fips-4602-feet/
# epsg=102749
# proj='+proj=lcc +lat_1=45.83333333333334 +lat_2=47.33333333333334 +lat_0=45.33333333333334 +lon_0=-120.5 +x_0=500000.0000000002 +y_0=0 +ellps=GRS80 +datum=NAD83 +to_meter=0.3048006096012192 +no_defs'


# get file names
fn_list=$(find ~/data/wadnr -type f -name *[0-9].tif)
# fn_list=$(ls baker_2015_dsm_[0-9]*.tif)

# convert feet to meters
parallel --progress "gdal_calc.py --co COMPRESS=LZW --co TILED=YES --co BIGTIFF=IF_SAFER --NoDataValue=-9999 --calc 'A*0.3048006096012192' -A {} --outfile {.}_m.tif" ::: $fn_list

# change projection from EPSG:102749 (NAD 1983 StatePlane Washington South FIPS 4602 Feet) to EPSG:26910 (NAD83 / UTM zone 10N)
parallel --progress "gdalwarp -co COMPRESS=LZW -co TILED=YES -co BIGTIFF=IF_SAFER -dstnodata -9999 -r cubic -s_srs EPSG:102749 -t_srs EPSG:26910 {} {.}_utm.tif" ::: *[0-9]_m.tif 

# convert from geoid to ellipsoid
parallel --progress "dem_geoid --reverse-adjustment {}" ::: *[0-9]_m_utm.tif

# edit to now in WGS 84 / UTM zone 10N
parallel --progress "gdal_edit.py -a_srs EPSG:32610 {}" ::: *[0-9]_m_utm-adj.tif

# build vrt to load as single file
gdalbuildvrt -vrtnodata -9999 -r cubic baker_2015_utm_m.vrt *[0-9]_m_utm-adj.tif

# create overviews for faster inspection
gdaladdo -r gauss -ro baker_2015_utm_m.vrt